In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#!ls /content/gdrive


Mounted at /content/drive


In [ ]:
# configuration

path = "/content/drive/My Drive/ECE324 Project/data/"
#path = "/content/gdrive/MyDrive/Colab Notebooks/"

CSV_PATH = '/content/drive/My Drive/ECE324 Project/data/images/usable_data.csv'
label_path = path + "labeled_recipe_name.csv"
vgg_path = path + "vgg_feature_vectors/"
resnet_path = path + "resnet_feature_vectors/"

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as Data
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image

if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)

Check for completeness of data. May be missing some pt files; catch and proceed as needed.

In [ ]:
# load labels
label_df = pd.read_csv(label_path)
label_df.pop('recipe_name')
label_df.rename(columns = {'url_ext':'code'}, inplace = True)
pd.set_option('display.max_columns', None)
#print(label_df.columns)
#print(label_df)

usable_data = pd.read_csv(CSV_PATH)
pd.set_option('display.max_columns', None)
#print(usable_data.columns)
#print(usable_data)

# check type for label_df['label'][0], check completeness of this data, convert or regenerate as needed
#print(type(label_df['label'][0]))
#print(len(label_df['label'][0]))

raw_df = pd.merge(label_df, usable_data['code'], how='inner', on=['code'])
pd.set_option('display.max_columns', None)
#print(raw_df)

train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()
train_name = path + "resnet_train_dataframe"
val_name = path + "resnet_val_dataframe"
test_name = path + "resnet_test_dataframe"

for batch in range(4):
  train_df = pd.DataFrame()
  val_df = pd.DataFrame()

  for i in range(batch*10000,(batch+1)*10000):
    resnet_features = torch.load(resnet_path+"resnet_"+str(raw_df.at[i,'code'])+".pt",map_location=torch.device('cpu'))
    current_label = raw_df['label'][i][1:-1].split(", ") #List of strings that should be floats
    current_label = list(map(lambda x: float(x), current_label))
    current_label = np.array(current_label)

    for j in range(5):
      train_df = train_df.append([[resnet_features[j].cpu().numpy(), current_label, raw_df.at[i,'code']]], ignore_index = True)
    val_df = val_df.append([[resnet_features[5].cpu().numpy(), current_label, raw_df.at[i,'code']]], ignore_index = True)

    if i % 100 == 0:
      print(i, "out of ", (batch+1)*10000)
      #train_df.to_pickle(train_name +str(batch)+".plk")
      #val_df.to_pickle(val_name +str(batch)+".plk")

  train_df.columns = ['resnet_features','label','code']
  val_df.columns = ['resnet_features','label','code']
  #train_df.to_pickle(train_name +str(batch)+".plk")
  #val_df.to_pickle(val_name +str(batch)+".plk")

#print(train_df['resnet_features'][1].shape)
#print(val_df)

for i in range(40000,raw_df.shape[0]):
  resnet_features = torch.load(resnet_path+"resnet_"+str(raw_df.at[i,'code'])+".pt",map_location=torch.device('cpu'))
  current_label = raw_df['label'][i][1:-1].split(", ") #List of strings that should be floats
  current_label = list(map(lambda x: float(x), current_label))
  current_label = np.array(current_label)

  for j in range(6):
    #test_df = test_df.append([[resnet_features[j].cpu().numpy(), current_label, raw_df.at[i,'code']]], ignore_index = True)

  if i % 100 == 0:
    print(i)
    #test_df.to_pickle(path + "resnet_test_dataframe.plk")

#test_df.columns = ['resnet_features','label','code']
#test_df.to_pickle(path + "resnet_test_dataframe.plk")

40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500


### Test Loading pickle files

In [ ]:
 t= pd.read_pickle(path + "resnet_val_dataframe2.plk")
 print(t)
 t= pd.read_pickle(path + "resnet_test_dataframe.plk")
print(t)

# convert to training dataloaders when needed

                                        resnet_features  \
0     [[0.38440582, 0.59222996, 0.40767357, 0.419694...   
1     [[0.39836776, 0.55181587, 0.4926117, 0.3743979...   
2     [[0.35347116, 0.56910765, 0.43696117, 0.399758...   
3     [[0.3648579, 0.49275362, 0.41715115, 0.3820423...   
4     [[0.39786085, 0.55153555, 0.52369845, 0.379644...   
...                                                 ...   
9995  [[0.36922157, 0.50380594, 0.47644514, 0.426244...   
9996  [[0.3636775, 0.47369605, 0.46837097, 0.4113553...   
9997  [[0.38720658, 0.50491893, 0.425788, 0.3899166,...   
9998  [[0.38399404, 0.4753618, 0.503357, 0.4022254, ...   
9999  [[0.37735242, 0.5433478, 0.5188493, 0.36758578...   

                                                  label    code  
0     [0.0033222591362126247, 0.0033222591362126247,...  216929  
1     [0.0033222591362126247, 0.0033222591362126247,...  216930  
2     [0.003322259136212624, 0.003322259136212624, 0...  216931  
3     [0.003322259136212625